In [33]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from numpy import count_nonzero

dir_ = '../data_0.5_track_20/'
# file_name = 'normalized_filter_track_5_user_100.csv'
# file_name = 'normalized_to_rating_filter_track_20_user_100.csv'
file_name = 'normalized_popularity_filter_track_20_user_100.csv'

In [34]:
prediction_count = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_count_top_N_' + file_name[:-3] + 'pkl'))
prediction_rating = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_rating_top_N_' + file_name[:-3] + 'pkl'))

In [35]:
file_name = 'normalized_to_rating_filter_track_20_user_100.csv'
pd_train = pd.read_pickle(os.path.join(dir_, 'train_' + file_name[:-3] + 'pkl'))

In [36]:
pd_train

,uid,tid,rating
761964,143,2600,0.25
942432,163,6175,1.75
294180,60,5301,3.75
999306,170,73,1.00
339170,70,4005,3.75
...,...,...,...
383800,73,1428,4.50
280364,57,8573,3.00
206333,42,3530,3.00
663060,126,3806,0.25


In [37]:
prediction_count

,tid,count,rating
68,68,3150,408.00
224,224,1924,131.50
6216,6216,1572,59.75
28,28,1388,264.50
4908,4908,1352,37.25
...,...,...,...
4098,4098,7,4.25
7246,7246,6,1.50
6807,6807,6,5.75
7119,7119,5,2.25


In [38]:
prediction_rating

,tid,count,rating
68,68,3150,408.00
28,28,1388,264.50
169,169,1152,225.00
668,668,1187,217.75
496,496,800,215.00
...,...,...,...
5258,5258,9,3.00
7119,7119,5,2.25
6814,6814,9,2.25
7246,7246,6,1.50


In [39]:
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])
test

,uid,tid,rating
936327,162,8410,0.75
261254,51,5512,4.75
311557,65,2018,0.25
102036,17,5634,2.50
397867,74,1053,3.00
...,...,...,...
447006,86,644,3.25
1193620,193,8585,4.00
1088496,180,6548,3.75
344501,71,577,1.75


In [40]:
test

,uid,tid,rating
936327,162,8410,0.75
261254,51,5512,4.75
311557,65,2018,0.25
102036,17,5634,2.50
397867,74,1053,3.00
...,...,...,...
447006,86,644,3.25
1193620,193,8585,4.00
1088496,180,6548,3.75
344501,71,577,1.75


In [41]:
num_user = 211
num_user

211

In [326]:
#############################################
#                                           #
#          Personal Recommendation          #
#                                           #
#############################################

In [22]:
n = 20

evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user)):
    precision_r = 0
    precision_c = 0
    
    top_n_list_r = []
    top_n_list_c = []
    
    u = pd_train[pd_train['uid']==i]

    for _, row in prediction_rating.iterrows():
        t = u[u['tid']== row[0]]
        if len(t) == 0:
            top_n_list_r.append(row[0])
        if len(top_n_list_r) == n:
            break
            
    for _, row in prediction_count.iterrows():
        t = u[u['tid']== row[0]]
        if len(t) == 0:
            top_n_list_c.append(row[0])
        if len(top_n_list_c) == n:
            break
    
    for j in top_n_list_r:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision_r += 1
            
    for j in top_n_list_c:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision_c += 1    
            
    satisfication_r += precision_r / n
    satisfication_c += precision_c / n

    
    evaluation_r.append(precision_r)
    evaluation_c.append(precision_c)

#     print(precision, satisfication)
print('rating', satisfication_r/num_user)
print('count',satisfication_c/num_user)


rating 0.4033175355450238
count 0.26919431279620853


In [ ]:
#############################################
#                                           #
#          Group Recommendation             #
#                                           #
#############################################

In [ ]:
groups = []
group_size = 2

for i in range(num_user//group_size):
    groups.append([])
    
for i in range(num_user):
    b = True
    while b:
        g = random.random()*(num_user/group_size)//1
        group = groups[int(g)]
        if len(group) < group_size:
            group.append(i)
            b = False
groups

In [ ]:
# check if duplicate 

users = []
for group in groups:
    for u in group:
        users.append(u)
users.sort(reverse=True)

n = num_user - 1 
for i in users:
    if i-n != 0:
        print('False')
    n-=1

In [ ]:
#############################################
#                                           #
#          Group Recommendation (2)         #
#                                           #
#############################################

In [42]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_2_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([144., 151.])

In [49]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    top_n_list_r = []
    top_n_list_c = []
    
    u1 = pd_train[pd_train['uid']==group[0]]
    u2 = pd_train[pd_train['uid']==group[1]]

    for _, row in prediction_rating.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        s = len(t1) + len(t2)
        if s == 0:
            top_n_list_r.append(row[0])
        if len(top_n_list_r) == n:
            break
            
    for _, row in prediction_count.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        s = len(t1) + len(t2)
        if s == 0:
            top_n_list_c.append(row[0])
        if len(top_n_list_c) == n:
            break
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
#         if s > 0:
#             precision_r += 1
        precision_r += math.log(s+1,10)/math.log(len(group)+1,10)
    print(math.log(s+1,10)/math.log(len(group+1),10))
    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]

        s = len(t1) + len(t2)
        if s > 0:
            precision_c += 1
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))

0.0
1.0
1.0
1.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.5849625007211565
1.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
1.5849625007211565
0.0
0.0
1.0
1.0
0.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
1.5849625007211565
0.0
0.0
0.0
0.0
1.0
1.5849625007211565
1.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
1.0
0.0
1.0
0.0
1.0
1.0
1.0
1.0
0.0
1.5849625007211565
0.0
1.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.5849625007211565
1.0
1.0
1.0
0.0
1.0
0.0
0.0
1.0
0.0

rating 0.4397632434830057
count 0.42857142857142844


In [44]:
math.log(2,10)

0.30102999566398114

In [ ]:
#############################################
#                                           #
#          Group Recommendation (4)         #
#                                           #
#############################################

In [25]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_4_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([  3.,  38.,  89., 130.])

In [26]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    top_n_list_r = []
    top_n_list_c = []
    
    u1 = pd_train[pd_train['uid']==group[0]]
    u2 = pd_train[pd_train['uid']==group[1]]
    u3 = pd_train[pd_train['uid']==group[2]]
    u4 = pd_train[pd_train['uid']==group[3]]

    for _, row in prediction_rating.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        t3 = u3[u3['tid']== row[0]]
        t4 = u4[u4['tid']== row[0]]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s == 0:
            top_n_list_r.append(row[0])
        if len(top_n_list_r) == n:
            break
            
    for _, row in prediction_count.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        t3 = u3[u3['tid']== row[0]]
        t4 = u4[u4['tid']== row[0]]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s == 0:
            top_n_list_c.append(row[0])
        if len(top_n_list_c) == n:
            break
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            precision_r += 1
    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4)
        if s > 0:
            precision_c += 1
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))


rating 0.8086538461538461
count 0.5903846153846155


In [ ]:
#############################################
#                                           #
#          Group Recommendation (5)         #
#                                           #
#############################################

In [27]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_5_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 17.,  31.,  65.,  92., 154.])

In [28]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    top_n_list_r = []
    top_n_list_c = []
    
    u1 = pd_train[pd_train['uid']==group[0]]
    u2 = pd_train[pd_train['uid']==group[1]]
    u3 = pd_train[pd_train['uid']==group[2]]
    u4 = pd_train[pd_train['uid']==group[3]]
    u5 = pd_train[pd_train['uid']==group[4]]


    for _, row in prediction_rating.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        t3 = u3[u3['tid']== row[0]]
        t4 = u4[u4['tid']== row[0]]
        t5 = u5[u5['tid']== row[0]]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s == 0:
            top_n_list_r.append(row[0])
        if len(top_n_list_r) == n:
            break
            
    for _, row in prediction_count.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        t3 = u3[u3['tid']== row[0]]
        t4 = u4[u4['tid']== row[0]]
        t5 = u5[u5['tid']== row[0]]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s == 0:
            top_n_list_c.append(row[0])
        if len(top_n_list_c) == n:
            break
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            precision_r += 1
    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5)
        if s > 0:
            precision_c += 1
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))


rating 0.8595238095238092
count 0.6297619047619049


In [ ]:
#############################################
#                                           #
#          Group Recommendation (10)        #
#                                           #
#############################################

In [29]:
# np.savetxt(os.path.join(dir_, 'groups_10_' + file_name[:-3] + 'csv'), groups, delimiter=",")
groups = np.loadtxt(os.path.join(dir_, 'groups_10_' + file_name), delimiter=',')
group_size = len(groups[0])
groups[0]

array([ 28.,  51.,  73.,  87.,  99., 113., 117., 118., 136., 137.])

In [30]:
evaluation_r = []
evaluation_c = []

satisfication_r = 0
satisfication_c = 0

for i in tqdm.tqdm_notebook(range(num_user//group_size)):
    group = groups[i]    
    precision_r = 0
    precision_c = 0
    
    top_n_list_r = []
    top_n_list_c = []
    
    u1 = pd_train[pd_train['uid']==group[0]]
    u2 = pd_train[pd_train['uid']==group[1]]
    u3 = pd_train[pd_train['uid']==group[2]]
    u4 = pd_train[pd_train['uid']==group[3]]
    u5 = pd_train[pd_train['uid']==group[4]]
    u6 = pd_train[pd_train['uid']==group[5]]
    u7 = pd_train[pd_train['uid']==group[6]]
    u8 = pd_train[pd_train['uid']==group[7]]
    u9 = pd_train[pd_train['uid']==group[8]]
    u10 = pd_train[pd_train['uid']==group[9]]

    for _, row in prediction_rating.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        t3 = u3[u3['tid']== row[0]]
        t4 = u4[u4['tid']== row[0]]
        t5 = u5[u5['tid']== row[0]]
        t6 = u6[u6['tid']== row[0]]
        t7 = u7[u7['tid']== row[0]]
        t8 = u8[u8['tid']== row[0]]
        t9 = u9[u9['tid']== row[0]]
        t10 = u10[u10['tid']== row[0]]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s == 0:
            top_n_list_r.append(row[0])
        if len(top_n_list_r) == n:
            break
            
    for _, row in prediction_count.iterrows():
        t1 = u1[u1['tid']== row[0]]
        t2 = u2[u2['tid']== row[0]]
        t3 = u3[u3['tid']== row[0]]
        t4 = u4[u4['tid']== row[0]]
        t5 = u5[u5['tid']== row[0]]
        t6 = u6[u6['tid']== row[0]]
        t7 = u7[u7['tid']== row[0]]
        t8 = u8[u8['tid']== row[0]]
        t9 = u9[u9['tid']== row[0]]
        t10 = u10[u10['tid']== row[0]]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s == 0:
            top_n_list_c.append(row[0])
        if len(top_n_list_c) == n:
            break
    
    for j in top_n_list_r:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            precision_r += 1
    satisfication_r += precision_r / n
    evaluation_r.append(precision_r)
    
    for j in top_n_list_c:
        t1 = test[test['uid'] == group[0]]
        t2 = test[test['uid'] == group[1]]
        t3 = test[test['uid'] == group[2]]
        t4 = test[test['uid'] == group[3]]
        t5 = test[test['uid'] == group[4]]
        t6 = test[test['uid'] == group[5]]
        t7 = test[test['uid'] == group[6]]
        t8 = test[test['uid'] == group[7]]
        t9 = test[test['uid'] == group[8]]
        t10 = test[test['uid'] == group[9]]
        
        t1 = t1[t1['tid']==j]
        t2 = t2[t2['tid']==j]
        t3 = t3[t3['tid']==j]
        t4 = t4[t4['tid']==j]
        t5 = t5[t5['tid']==j]
        t6 = t6[t6['tid']==j]
        t7 = t7[t7['tid']==j]
        t8 = t8[t8['tid']==j]
        t9 = t9[t9['tid']==j]
        t10 = t10[t10['tid']==j]
        s = len(t1) + len(t2) + len(t3) + len(t4) + len(t5) + len(t6) + len(t7) + len(t8) + len(t9) + len(t10)
        if s > 0:
            precision_c += 1
    satisfication_c += precision_c / n
    evaluation_c.append(precision_c)
    
print('rating', satisfication_r/(num_user//group_size))
print('count', satisfication_c/(num_user//group_size))


rating 0.9571428571428571
count 0.7761904761904761


In [185]:
#############################################
#                                           #
#          Distribution of rating           #
#                                           #
#############################################

In [317]:
dir_ = '../data_0.5_track_20/'
# file_name = 'normalized_filter_track_5_user_100.csv'
file_name = 'normalized_to_rating_filter_track_20_user_100.csv'

In [318]:
ratings = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
rating_list = []

for i in range(1,6):
    rating = ratings[ratings['rating']<=i] 
    num = len(rating)
    rating_list.append(num)
    print(num)
    
print('')

for i in range(4,0,-1):
    rating_list[i] -= rating_list[i-1]
    
for i in range(5):
    print(rating_list[i])

111228
149490
196414
243281
296137

111228
38262
46924
46867
52856
